In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('C:/Users/ASUS/Downloads/data (1).csv')

## Main codes used

In [3]:
pd.set_option('display.max_rows',None)

In [4]:
df.isna().sum()

BPrev                                                       0
BStreak                                                     0
B_Age                                                       3
B_Height                                                    1
B_HomeTown                                                  6
B_ID                                                        0
B_Location                                                  7
B_Name                                                      0
B_Weight                                                    0
B__Round1_Grappling_Reversals_Landed                      499
B__Round1_Grappling_Standups_Landed                       499
B__Round1_Grappling_Submissions_Attempts                  499
B__Round1_Grappling_Takedowns_Attempts                    499
B__Round1_Grappling_Takedowns_Landed                      499
B__Round1_Strikes_Body Significant Strikes_Attempts       499
B__Round1_Strikes_Body Significant Strikes_Landed         499
B__Round

In [5]:
# these are not continous, => not nesseary to handle NAs
list(set(df.columns) - set(df.describe().columns))

['R_Location',
 'winby',
 'B_Location',
 'R_HomeTown',
 'B_Name',
 'R_Name',
 'Date',
 'B_HomeTown',
 'winner']

In [6]:
# helper
my_list = list(df.describe().columns)

list_item_to_remove = ['BPrev','BStreak','B_Age','B_Height','B_ID','B_Weight','Event_ID','Fight_ID','Last_round'
                       ,'Max_round','RPrev','RStreak','R_Age','R_Height','R_ID','R_Weight']
 
result_lst = list(set(my_list) - set(list_item_to_remove))
result_lst

['B__Round2_TIP_Side Control Time',
 'B__Round4_TIP_Ground Time',
 'B__Round4_Strikes_Ground Total Strikes_Attempts',
 'R__Round3_Strikes_Clinch Total Strikes_Attempts',
 'R__Round5_Strikes_Clinch Significant Strikes_Landed',
 'B__Round1_Grappling_Takedowns_Landed',
 'R__Round3_Strikes_Distance Head Punches_Landed',
 'B__Round4_Strikes_Body Total Strikes_Attempts',
 'R__Round4_Strikes_Distance Strikes_Attempts',
 'R__Round1_Strikes_Clinch Body Strikes_Attempts',
 'R__Round5_Strikes_Ground Leg Strikes_Attempts',
 'B__Round2_Strikes_Ground Significant Kicks_Attempts',
 'B__Round2_TIP_Clinch Time',
 'R__Round2_Strikes_Ground Significant Kicks_Attempts',
 'R__Round1_Strikes_Ground Total Strikes_Attempts',
 'B__Round4_TIP_Clinch Time',
 'B__Round5_TIP_Back Control Time',
 'B__Round3_Strikes_Distance Head Strikes_Landed',
 'B__Round2_Strikes_Distance Head Kicks_Landed',
 'R__Round4_Strikes_Legs Total Strikes_Attempts',
 'R__Round2_Strikes_Clinch Significant Strikes_Attempts',
 'R__Round5_Str

In [7]:
# droping all empty rows
df['Sum'] = df[result_lst].sum(axis = 1)
df = df[df.Sum != 0]
df.shape

(1235, 896)

In [8]:
# droping all columns with less than 85% non NA values (1235 * 85% = 1050)
df.dropna(thresh=1050, inplace=True, axis=1)
df.shape
df.isna().sum()

BPrev                                                      0
BStreak                                                    0
B_Age                                                      3
B_Height                                                   1
B_HomeTown                                                 6
B_ID                                                       0
B_Location                                                 6
B_Name                                                     0
B_Weight                                                   0
Date                                                       0
Event_ID                                                   0
Fight_ID                                                   0
Last_round                                                 0
Max_round                                                  0
RPrev                                                      0
RStreak                                                    0
R_Age                   

In [9]:
# I see a lot of columns with only 100 missing values so decided to remove the rows but not the columns
df.dropna(inplace=True)
df.shape

(1035, 111)

In [10]:
# When you clean the categorical variables, keep the top 70% of the values, and assign the 30% to a new value ‘other’.

# I'll do this with one example, but should do the same for R_HomeTown, B_Location and R_Location columns as well.
b_city_freq = (df['B_HomeTown'].value_counts())*100/df.shape[0]
botton_decile = b_city_freq.quantile(q=0.3)
less_freq_cities = b_city_freq[b_city_freq<=botton_decile]
df.loc[df['B_HomeTown'].isin(less_freq_cities.index.tolist()), 'B_HomeTown'] = "other"

In [11]:
# Bin the data for columns ‘R_Weight’ and ‘B_Weight’.

bins = [0, 50, 70, 90, 100]
df['Binned_R_Weight'] = pd.cut(df['R_Weight'], bins)
df['Binned_B_Weight'] = pd.cut(df['B_Weight'], bins)

In [12]:
# Turn into dummy variables (I did for 'winner' - could be useful for logistic regression, for example)

df_fin = pd.get_dummies(df, columns=['winner'])

In [13]:
# Do not include unique indicator columns, such as the name of the fighters

df_fin = df_fin.drop(columns=['B_Name', 'Date','B_ID','R_Name','R_ID','Sum'])

In [14]:
# Split the data into test and train

train, test = train_test_split(df_fin)

In [15]:
# Save the data into train.csv and test.csv 

train.to_csv(r'C:/Users/ASUS/Downloads/train.csv')
test.to_csv(r'C:/Users/ASUS/Downloads/test.csv')

## What else I used beforehand

In [16]:
df.shape

(1035, 113)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1035 entries, 0 to 1475
Columns: 113 entries, BPrev to Binned_B_Weight
dtypes: category(2), float64(89), int64(13), object(9)
memory usage: 908.1+ KB


In [18]:
df.columns

Index(['BPrev', 'BStreak', 'B_Age', 'B_Height', 'B_HomeTown', 'B_ID',
       'B_Location', 'B_Name', 'B_Weight', 'Date',
       ...
       'R__Round1_TIP_Misc. Ground Control Time',
       'R__Round1_TIP_Mount Control Time', 'R__Round1_TIP_Neutral Time',
       'R__Round1_TIP_Side Control Time', 'R__Round1_TIP_Standing Time',
       'winby', 'winner', 'Sum', 'Binned_R_Weight', 'Binned_B_Weight'],
      dtype='object', length=113)

In [19]:
df.describe(include=["object", "bool"])

,B_HomeTown,B_Location,B_Name,Date,R_HomeTown,R_Location,R_Name,winby,winner
count,1035,1035,1035,1035,1035,1035,1035,1035,1035
unique,264,369,576,181,402,320,494,3,3
top,other,Rio de Janeiro Brazil,Paul Felder,12/14/2015,Rio de Janeiro Brazil,Rio de Janeiro Brazil,Donald Cerrone,DEC,red
freq,209,23,6,26,32,46,9,495,593


In [20]:
df['B_HomeTown'].value_counts() 
# or I could use len(df['B_HomeTown'].unique())
df['B_Location'].value_counts() 
df['R_HomeTown'].value_counts() 
df['R_Location'].value_counts() 
df['B_Name'].value_counts() # not unique, => I would keep (but will remove as specified in the task)
df['R_Name'].value_counts() 
df['B_ID'].value_counts() 

2070    6
1952    6
2316    6
2205    5
1272    5
1806    5
1944    5
2168    5
2122    5
2200    5
2057    5
1340    5
2172    4
2469    4
1984    4
1866    4
890     4
2297    4
2299    4
2230    4
2157    4
1966    4
2429    4
1481    4
2371    4
372     4
2309    4
2724    4
2095    4
2263    4
2073    4
2778    4
2119    4
2291    4
1550    4
1531    4
2610    4
2603    4
2296    4
545     4
2589    4
2585    4
2053    4
2264    4
1347    3
2281    3
2322    3
2323    3
2287    3
1515    3
2312    3
2290    3
1422    3
2300    3
1499    3
1504    3
1511    3
368     3
2332    3
502     3
750     3
2756    3
2748    3
2813    3
2731    3
648     3
2678    3
786     3
2602    3
808     3
515     3
509     3
2529    3
538     3
449     3
435     3
432     3
2466    3
1061    3
1694    3
367     3
347     3
377     3
2373    3
735     3
2352    3
2276    3
2047    3
2166    3
2238    3
2105    3
2204    3
2208    3
2118    3
2010    3
1978    3
1875    3
2087    3
2215    3
1991    3


In [21]:
df1 = df.drop(list(set(df.columns) - set(df.describe().columns)), axis='columns')
df1.drop(['B_ID','R_ID'], axis='columns', inplace = True)
df1['Total'] = df.sum(axis=1)

df2 = df[list(set(df.columns) - set(df.describe().columns))]

duplicateDFRow = df2[df2.duplicated()]
print(duplicateDFRow)

Empty DataFrame
Columns: [R_Location, winby, Date, B_Location, Binned_B_Weight, R_HomeTown, B_Name, Binned_R_Weight, R_Name, B_HomeTown, winner]
Index: []


In [22]:
df[df.describe().columns]

,BPrev,BStreak,B_Age,B_Height,B_ID,B_Weight,Event_ID,Fight_ID,Last_round,Max_round,...,R__Round1_TIP_Ground Control Time,R__Round1_TIP_Ground Time,R__Round1_TIP_Guard Control Time,R__Round1_TIP_Half Guard Control Time,R__Round1_TIP_Misc. Ground Control Time,R__Round1_TIP_Mount Control Time,R__Round1_TIP_Neutral Time,R__Round1_TIP_Side Control Time,R__Round1_TIP_Standing Time,Sum
0,1,1,23.0,182.0,2783,84,802,6405,3,3,...,432.0,674.0,35.0,42.0,0.0,94.0,554.0,0.0,862.0,17878.0
2,2,0,38.0,172.0,721,70,729,5523,1,3,...,97.0,143.0,0.0,0.0,11.0,0.0,374.0,0.0,459.0,10228.0
3,0,0,23.0,170.0,2825,56,790,6240,1,3,...,515.0,900.0,102.0,197.0,0.0,176.0,651.0,22.0,900.0,21384.0
4,3,1,30.0,167.0,2260,61,785,6226,3,3,...,8.0,338.0,0.0,0.0,8.0,0.0,654.0,0.0,1162.0,25003.0
5,0,0,38.0,180.0,1918,70,745,5655,3,3,...,100.0,100.0,0.0,25.0,0.0,58.0,874.0,0.0,1103.0,13995.0
6,2,1,30.0,182.0,2507,70,765,5914,3,3,...,11.0,12.0,0.0,0.0,0.0,3.0,553.0,0.0,673.0,14166.0
8,1,0,34.0,177.0,1817,70,717,5370,3,3,...,0.0,166.0,0.0,0.0,0.0,0.0,655.0,0.0,711.0,10421.0
9,2,1,33.0,182.0,2080,77,766,5972,3,3,...,91.0,91.0,0.0,13.0,0.0,29.0,413.0,0.0,648.0,17975.0
13,0,0,44.0,177.0,1001,84,679,4885,4,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1150.0
14,3,3,30.0,190.0,2608,93,800,6362,1,3,...,0.0,37.0,0.0,0.0,0.0,0.0,332.0,0.0,564.0,11671.0
